# Slagalica


Problem rasporedjivanja elemenata
```
1 2 3 4
5 6   7
8 9 A B
C D E F
```
tako da na kraju imaju oblik

```
1 2 3 4
5 6 7 8
9 A B C
D E F 

```


Kao i kod osalih primera se kreće dizajnom stanja problema. U ovom slučaju to će biti 
matrica.

Heuristička funkcija kojom merimo koliko je neko stanje dobro u ovom slučaju će biti broj elemenata koji se nalaze na dobroj poziciji.

## State

In [208]:
class State:
    """
    A = [[1, 2, 3, 4],
         [5, 6, 0, 7],
         [8, 9, 10, 11],
         [12, 13, 14, 15]]    
    """
    def __init__(self, A):
        self.tabla = A
        
    def h(self):
        t = 0
        c = 1
        for i in range(4):
            for j in range(4):
                #print c, self.tabla[i][j], t
                if self.tabla[i][j]==c:
                    t+=1
                c += 1
        if self.tabla[3][3] == 0:
            t+=1
        return t
        
    def __str__(self):
        A = self.tabla
        s = ""
        for i in range(4):
            sep = ""
            for j in range(4):
                s += sep + '{:>4}'.format(A[i][j])
                sep = ", "
            if i!=3:
                s += "\n"
        return s

### Test

Najbolje je ako se odmah proveri da li osnovne funkcije rade kako treba

In [216]:
A = [[1, 2, 3, 4],
     [5, 6, 0, 7],
     [8, 9, 10, 11],
     [12, 13, 14, 15]]    

s = State(A)
print s
print s.h()

   1,    2,    3,    4
   5,    6,    0,    7
   8,    9,   10,   11
  12,   13,   14,   15
6


Sledeći korak je dizajn klase kojom opisujemo sam problem

### Problem

Potrebno je definisati funkcije
- isGoal
- firstState
- akcije


In [218]:
from copy import deepcopy

class Problem:
    def isGoal(self, state):
        t = state.h()
        if t == 16:
            return True
        else:
            return False
    
    def firstState(self):
        A = [[1, 2, 3, 4],
             [5, 0, 6, 8],
             [9, 10, 7, 11],
             [13, 14, 15, 12]]    
        return State(A)
    
    def prazno(self, state):
        for i in range(4):
            for j in range(4):
                if(state.tabla[i][j] == 0):
                    return (i, j)
        return (-1, -1)
    
    def akcije(self, state):
        ret = []
        smerovi = [('gore', 1, 0), ('dole', -1, 0), ('levo', 0, 1), ('desno', 0, -1)]
        (i0, j0) = self.prazno(state)
        for sm in smerovi:
            ii = i0 + sm[1]
            jj = j0 + sm[2]
            if(ii>=0 and ii<=3 and jj>=0 and jj<=3):
                s = deepcopy(state)
                s.tabla[i0][j0] = s.tabla[ii][jj]
                s.tabla[ii][jj] = 0
                ret.append((sm[0], s))                    
        return ret
In [ ]:


### Node

Klasa Node u sebi sadrži neophodne elemente za formiranje stabla pretrage.

In [220]:
class Node:
    def __init__(self, parent, action, state):
        self.parent = parent
        self.action = action
        self.state = state
        
    def __str__(self):
        return str(self.state)
        
    def children(self):
        max_h = h
        akcije = problem.akcije(self.state)
        ret = []
        for ak in akcije:
            nnode = Node(self, ak[0], ak[1])
            ret.append(nnode)
        return ret


#### Test
Još malo provere da li prethodno radi kako treba.

In [221]:
problem = Problem()    
start = problem.firstState()
print 'prazno:', problem.prazno(start)

start_node = Node(None, None, start)

children = start_node.children()
for child in children:
    print '--'*20
    print child
    print child.action
    print child.state.h()


prazno: (1, 1)
----------------------------------------
   1,    2,    3,    4
   5,   10,    6,    8
   9,    0,    7,   11
  13,   14,   15,   12
gore
10
----------------------------------------
   1,    0,    3,    4
   5,    2,    6,    8
   9,   10,    7,   11
  13,   14,   15,   12
dole
10
----------------------------------------
   1,    2,    3,    4
   5,    6,    0,    8
   9,   10,    7,   11
  13,   14,   15,   12
levo
12
----------------------------------------
   1,    2,    3,    4
   0,    5,    6,    8
   9,   10,    7,   11
  13,   14,   15,   12
desno
10


### Algoritam

U ovoj implementaciji možete da menjate vrednost parametara:

- slepa: True, False
- strategija: 'bfs', 'dfs'

In [222]:
slepa = False
strategija = 'bfs'

problem = Problem()    
start = problem.firstState()
start_node = Node(None, None, start)

front = [start_node]
seen = {}
count = 0
max_front = 1
while(len(front)>0):   
    if(slepa):
        node = front.pop(0)        
    else:
        max_h = front[0].state.h()
        max_el = 0
        for i in range(len(front)):
            h = front[i].state.h()
            if h>max_h:
                max_h = h
                max_el = i
        node = front.pop(max_el)        
    
    if problem.isGoal(node.state):
        print 'reseno'
        t = node
        while t.parent != None:
            print t.state
            print '..........'+t.action
            t = t.parent
        print start_node
        break
    
    max_front = max([max_front, len(front)])    
    key = str(node.state)
    if(key not in seen):
        seen[key] = node
        children = node.children()
        for child in children:
            if strategija == 'dfs':
                front.insert(0, child) # dodavanje na pocetak
            else:
                front.append(child) # dodavanje na kraj
    count += 1
    if(count %100 == 0):
        print count, len(front), max_h
print 'Broj koraka: {0}'.format(count)
print 'Max velicina fronta: {0}'.format(max_front)

reseno
   1,    2,    3,    4
   5,    6,    7,    8
   9,   10,   11,   12
  13,   14,   15,    0
..........gore
   1,    2,    3,    4
   5,    6,    7,    8
   9,   10,   11,    0
  13,   14,   15,   12
..........levo
   1,    2,    3,    4
   5,    6,    7,    8
   9,   10,    0,   11
  13,   14,   15,   12
..........gore
   1,    2,    3,    4
   5,    6,    0,    8
   9,   10,    7,   11
  13,   14,   15,   12
..........levo
   1,    2,    3,    4
   5,    0,    6,    8
   9,   10,    7,   11
  13,   14,   15,   12
Broj koraka: 4
Max velicina fronta: 9


## Domaći

### 1. Lavirint

Ako je dat lavirint
```
+--+---------+-----------------+---------+
|  |         |                 | C       |
|  +  +----  +-------   +------+------   |
|   A |                 |                |
|  +--+-------------+   +----+   +-------+
|                   |                    |
|  +-------------   +------------+   +---+
|                                        |
+----------------------------------------+
```
a) Pronaći najkraći put

b) Pronaći sve puteve



### 2. Sistem linearnih algebarskih jednačina

Ako je dat sistem algebarskih jednačina:

$$ \begin{array}{}
  a_{1,1}x_1+a_{1,2}x_2+a_{1,3}x_3+a_{1,4}x_4 = b_1 \\
  a_{2,1}x_1+a_{2,2}x_2+a_{2,3}x_3+a_{2,4}x_4 = b_2 \\
  a_{3,1}x_1+a_{3,2}x_2+a_{3,3}x_3+a_{3,4}x_4 = b_3 \\
  a_{4,1}x_1+a_{4,2}x_2+a_{4,3}x_3+a_{4,4}x_4 = b_4
  \end{array}
$$

Pronaći približne vrednosti za $x_1, x_2, x_3, x_4$ takve da budu rešenje sistema.
